# Initialize the client

In [1]:
import requests
from bs4 import BeautifulSoup
from openai import OpenAI
import validators

Defining  Function which Fetch Site Content

In [2]:
def fetch_site_content(url):
    if not validators.url(url):
        raise ValueError("Invalid URL provided.")
    response = requests.get(url, timeout=10)
    response.raise_for_status()
    return response.text

extract site info

In [3]:
def extract_site_info(html):
    soup = BeautifulSoup(html, "html.parser")
    title = soup.title.string if soup.title else "No title"
    meta_desc = soup.find("meta", attrs={"name": "description"})
    description = meta_desc["content"] if meta_desc else "No description"
    text_content = " ".join([p.get_text() for p in soup.find_all("p")])
    return {
        "title": title,
        "description": description,
        "text_content": text_content[:2000]  # limit for prompt
    }

Generate Audit

In [4]:
def generate_audit(site_info, url):
    from openai import OpenAI
    client = OpenAI()

    prompt = f"""
    Perform a professional creative audit for the website: {url}.
    Analyze the following aspects:
    - Design (layout, typography, colors, accessibility)
    - Content (clarity, tone, SEO, readability)
    - UX (navigation, responsiveness, speed)
    - Branding (consistency, emotional impact, differentiation)

    Website Title: {site_info['title']}
    Meta Description: {site_info['description']}
    Sample Content: {site_info['text_content']}

    Perform a Creative Audit on this page. Structure your response into:"
        "1. **Blink Test**: Is the value prop clear in 3 seconds?"
        "2. **Visual Hierarchy**: Evaluate the use of 'Golden Ratio' and focal points."
        "3. **Color & Typography**: Do they match the brand's industry authority?"
        "4. **The Friction List**: What design elements are 'ugly' or distracting?"
        "5. **Creative Score**: Rate out of 100.

    Provide strengths, weaknesses, and recommendations in a structured format.
    """

    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": "You are a professional creative consultant."},
            {"role": "user", "content": prompt},
        ],
        max_tokens=1000
    )

    # ✅ Correct way to access the content
    return response.choices[0].message.content

Do Creative Audit Call

In [5]:
def creative_audit_tool(url):
    html = fetch_site_content(url)
    site_info = extract_site_info(html)
    audit_report = generate_audit(site_info, url)
    return audit_report

Main Function

In [ ]:
if __name__ == "__main__":
    target_url = input("Enter the website URL to audit: ")
    if not target_url.startswith("http"):
        target_url = "https://" + target_url
        
    audit_report = creative_audit_tool(target_url)
    print("\n" + "="*50 + "\nCREATIVE AUDIT REPORT\n" + "="*50)
    print(audit_report)
